# Description
In this notebook methods to find the ags for factories.

In [ ]:
import pandas as pd
import numpy as np

# Load data

In [ ]:
def add_zeros(ags):
    if not pd.isna(ags):
        ags = str(ags)
        length_ags = len(ags)
        if length_ags < 8:
            ags = "0" * (8 - length_ags) + ags
    return ags


df_ags = pd.read_csv("../data/public/ags/master.csv")

df_anlagen = pd.read_excel(
    "Anlagenliste_2018_DEHST.xlsx",
    sheet_name="Anlagen_Liste",
    decimal=",",
    thousands=".",
    converters={"Bundesland": str, "AGS_manual": str, "VET 2018 [t CO2 Äq]": int, "ID" : int},
)

df_anlagen.rename(
    columns={
        "Standort der Anlage": "Ort",
        "AGS": "AGS_xls",
        "VET 2018 [t CO2 Äq]": "VET2018_t",
        "Haupttätigkeit nach TEHG": "TEHG_Nr",
        "Bezeichnung Haupttätigkeit nach TEHG": "TEHG_name",
        "NACE WZ2008": "nace_wz",
    },
    inplace=True,
)
df_anlagen = df_anlagen[
    [
        "ID",
        "Nummer",
        "Betreiber",
        "Anlagenname",
        "Bundesland",
        "Ort Original",
        "Ort",
        "VET2018_t",
        "TEHG_Nr",
        "TEHG_name",
        "nace_wz",
        "AGS_manual",
    ]
]
df_anlagen["AGS_manual"] = df_anlagen["AGS_manual"].apply(add_zeros)
df_anlagen.dropna(subset=["nace_wz"], inplace=True)


# Data Processing
## Add industrie cateogires to facilities
Look at [Sharepoint](https://germanzero.sharepoint.com/:w:/r/Files/200_Campaigning_Mobilisierung/10_Klimaentscheide/06_LocalZero/03_Generator/20_KStP-Generator_v2/30_Industrie/70_Kommunenfeine_Bilanz/22-07-14_Methodischer%20Ansatz%20kommunenfeine%20Industrie.docx?d=wd6874aa0f4f944e19c6a29dda796c406&csf=1&web=1&e=SnSbBP) for further information about Nace

In [ ]:
# only usable for 2018 VET as not all NACE identifiers are added (only the ones present in the excel sheet)
mapper_nace_wz = {
    "miner_cement": ["23.51", "23.65"],
    "miner_chalk": ["23.52", "23.62"],
    "miner_glas": ["23.1"],
    "miner_ceram": ["23.2", "23.3", "23.4"],
    "chem_basic": ["20.11", "20.12", "20.13", "20.14", "20.16"],
    "chem_ammonia": ["20.15"],
    "chem_other": ["20.2", "20.3", "20.4", "20.5", "20.6", "21."],
    "metal_steel": ["24.1", "25.5", "25.61"],  # primary and secondary is sorted by cities later
    "metal_nonfe": ["24.4", "24.53", "24.54"],
    "other_paper": ["17."],
    "other_food": ["10.8", "10.6", "10.5", "10.4", "10.3", "11.05", "12.00"],
    # "other_further": ["8.", "22.", "24.2", "24.3", "25.", "28.1", "29.", "30."],
    "other_further": [
        "13.",
        "14.",
        "15.",
        "16.",
        "18.",
        "22.",
        "23.61",
        "23.64",
        "23.69",
        "23.7",
        "23.9" "24.2",
        "24.3",
        "25.1",
        "25.2",
        "25.3",
        "25.4",
        "25.62",
        "25.7",
        "25.8",
        "26.9",
        "27.",
        "28.1",
        "29.",
        "30.",
        "31.",
        "32.",
        "33.",
    ],
}
# steel_primariy_locations = [
#     "Bremen",
#     "Duisburg",
#     "Dillingen",
#     "Eisenhüttenstadt",
#     "Salzgitter",
#     "Völklingen",
# ]
ids_primary_steel = [56, 70, 52, 1086, 59, 53, 69, 206009, 60, 1228, 43] # manual selection of steel primary facilities


In [ ]:
df_anlagen["i_category"] = "No category"
for key, item in mapper_nace_wz.items():
    if len(item) > 0:
        for name in item:
            df_anlagen.loc[df_anlagen["nace_wz"].str.contains(name), "i_category"] = key

df_anlagen.loc[(df_anlagen["i_category"] == "metal_steel") & (df_anlagen["ID"].isin(ids_primary_steel)), "i_category"] = "metal_steel_primary"
df_anlagen.loc[df_anlagen["i_category"] == "metal_steel", "i_category"] = "metal_steel_secondary"

## Select only "Anlagen" from industry

In [ ]:
df_anlagen = df_anlagen[df_anlagen["i_category"] != "No category"]

In [ ]:
print(f"{len(df_anlagen)} selected")

## Add state information to df_ags

In [ ]:
ags_to_state = {
    "01" : "SH",
    "02" : "HH",
    "03" : "NI",
    "04" : "HB",
    "05" : "NW",
    "06" : "HE",
    "07" : "RP",
    "08" : "BW",
    "09" : "BY",
    "10" : "SL",
    "11" : "BE",
    "12" : "BB",
    "13" : "MV",
    "14" : "SN",
    "15" : "ST",
    "16" : "TH"
}

In [ ]:
df_ags["ags_state_digits"] = [x[:2] for x in df_ags["ags"]]
df_ags["state"] = df_ags["ags_state_digits"].map(ags_to_state)

## Remove not necessary parts of the Names in AGS

In [ ]:
# remove everything behind a , (for example `, Stadt` or `, Landkreis`)
df_ags["name"] = df_ags["description"].str.split(",").str[0]
# df_ags.to_csv("temp_python_export_ags_list.csv", index=False)

## Select duplicates in AGS List

In [ ]:
df_ags_duplicates = df_ags[df_ags.duplicated(subset="name", keep=False)]
df_ags_duplicates = df_ags_duplicates.sort_values("name")
df_ags_no_duplicates = df_ags.drop_duplicates(subset="name", keep=False)

In [ ]:
n = 0
mask_duplicates_state_unique = [False] * len(df_ags_duplicates)
while True:
    row = df_ags_duplicates.iloc[n]
    ort_count = df_ags_duplicates[df_ags_duplicates["name"] == row["name"]].shape[0]

    if ort_count ==2:
        if row["state"] != df_ags_duplicates.iloc[n + 1]["state"]:
            mask_duplicates_state_unique[n] = True
            mask_duplicates_state_unique[n + 1] = True

    elif ort_count ==3:
        if (
            (row["state"] != df_ags_duplicates.iloc[n + 1]["state"])
            & (
                row["state"]
                != df_ags_duplicates.loc[:, "state"].iloc[n + 2]
            )
            & (
                df_ags_duplicates.loc[:, "state"].iloc[n + 1]
                != df_ags_duplicates.loc[:, "state"].iloc[n + 2]
            )
        ):
            mask_duplicates_state_unique[n] = True
            mask_duplicates_state_unique[n + 1] = True
            mask_duplicates_state_unique[n + 2] = True

    n += ort_count
    if n >= len(df_ags_duplicates):
        break

In [ ]:
df_ags_duplicates_state_unique = df_ags_duplicates[mask_duplicates_state_unique]

# Find AGS for "Ort"
This is a multi step process iterating over every entry in df_anlagen terminating at the earliest match
1. if column AGS_manual has an entry this entry is used as AGS
2. search for a match of "Ort" inside the dg_ags_no_duplicates list
3. search for a match inside the df_ags_duplicates_state_unique for a match of "Ort" and "State"
4. For all not matched entries of df_anlagen check if the df_ags_no_duplicates "Ort" does start with the location of the "Anlage" (also check if the state matches)

In [ ]:
for index, row in df_anlagen.iterrows():
    state_anlage = row.loc["Bundesland"]
    location_anlage = row.loc["Ort"]
    if ~df_anlagen.loc[:,"AGS_manual"].isna().loc[index]:
        # print(df_anlagen.loc[index, "AGS_manual"])
        df_anlagen.loc[index, "AGS"] = df_anlagen.loc[index, "AGS_manual"]
        df_anlagen.loc[index, "match_type"] = "manual_entry"
    else:
        temp_ags_match = df_ags_no_duplicates.loc[(df_ags_no_duplicates["name"] == location_anlage)]
        # select entries with only one match
        if temp_ags_match.shape[0] == 1:
            if temp_ags_match["state"].values[0] == state_anlage:
                df_anlagen.loc[index, "AGS"] = temp_ags_match["ags"].values[0]
                df_anlagen.loc[index, "match_type"] = "direct"
        else:
            temp_ags_match_dup = df_ags_duplicates_state_unique.loc[(df_ags_duplicates_state_unique["name"] == location_anlage)]
            if temp_ags_match_dup.shape[0] == 1:
                raise ValueError("Duplicate entry selection wrong. Single entry found in AGS list")
            elif temp_ags_match_dup.shape[0] > 1:
                for index_match, match in temp_ags_match_dup.iterrows():
                    if match["state"] == state_anlage:
                        df_anlagen.loc[index, "AGS"] = match["ags"]
                        df_anlagen.loc[index, "match_type"] = "dublicate_match_state_unique"

for index, row in df_anlagen[df_anlagen["match_type"].isna()].iterrows():
    state_anlage = row.loc["Bundesland"]
    location_anlage = row.loc["Ort"]
    temp_ags_match = df_ags_no_duplicates[df_ags_no_duplicates["name"].str.startswith(location_anlage) == True]
    if temp_ags_match.shape[0] == 1:
        if temp_ags_match["state"].values[0] == state_anlage:
            df_anlagen.loc[index, "AGS"] = temp_ags_match["ags"].values[0]
            df_anlagen.loc[index, "match_type"] = "starts_with_only_one_match"

In [ ]:
df_anlagen[pd.isna(df_anlagen["match_type"])]

# Export results

In [ ]:
df_anlagen = df_anlagen[~pd.isna(df_anlagen["match_type"])]

In [ ]:
# df_anlagen[["Ort Original", "VET2018_t", "i_category", "AGS"]].to_csv("industry_ags_match_reduced_export.csv", index=False)

In [ ]:
# df_anlagen.to_csv("industry_ags_match_full_export.csv")

In [ ]:
# df_anlagen[~pd.isna(df_anlagen["match_type"])][df_anlagen["i_category"] == "other_further"][df_anlagen["nace_wz"].str.contains("22.1")]

## Group valuee by ags

In [ ]:
df_pivot = pd.pivot_table(
    df_anlagen,
    values="VET2018_t",
    index="AGS",
    columns="i_category",
    aggfunc=[np.sum, np.count_nonzero],
    margins=True,
    margins_name="DG000000"
).reset_index()
df_pivot.columns = ["_".join(a) for a in df_pivot.columns.to_flat_index()]
df_pivot.columns = df_pivot.columns.str.replace("sum_", "")
df_pivot.columns = df_pivot.columns.str.replace("AGS_", "ags")
df_pivot.columns = df_pivot.columns.str.replace("count_nonzero", "count")
df_pivot = df_pivot.set_index("ags", drop=True)
df_pivot = df_pivot.fillna(0.0)
# df_pivot.to_csv("data_exports/2018.csv", sep=",", decimal=".")


In [ ]:
df_pivot

# Comparsion with calculated values from NIR

In [ ]:
df = pd.read_csv(
    "i18_values_germany.csv",
    decimal=",",
    thousands=".",
    sep=";",
)
df[["cat", "type"]] = df["description"].str.split(pat=".", expand=True)[[2, 3]]
df.loc[:, "cat"] = df["cat"].str[2:]

In [ ]:
df_anlagen_sum = pd.DataFrame(df_anlagen[~pd.isna(df_anlagen["match_type"])].groupby("i_category")["VET2018_t"].sum().sort_values(ascending=False))

In [ ]:
df_merged = pd.merge(df_anlagen_sum, df[df["type"] == "CO2e_total"],left_index=True, right_on="cat" )
df_merged.drop(columns=["description", "type"], inplace=True)
df_merged.rename(columns={"value": "NIR_CO2e"}, inplace=True)
df_merged["DEHST/NIR"] = df_merged["VET2018_t"] / df_merged["NIR_CO2e"]
df_merged.sort_values(by="cat", inplace=True)

In [ ]:
print(df_merged["NIR_CO2e"].sum(), df_merged["VET2018_t"].sum())

In [ ]:
df_merged

In [ ]:
# label,group,description,value,unit,rationale,reference,link
df_temp = df_merged[["cat", "DEHST/NIR"]].reset_index(drop=True).copy()
df_facts = pd.DataFrame()
df_facts["label"] = "Fact_I_P_" + df_temp["cat"] + "_ratio_CO2e_excel_to_dehst"
df_facts["group"] = "ui"
df_facts["description"] = "Ratio of CO2e calculated based on NIR to the sum of emissions from industry plants included in DEHST."
df_facts["value"] = df_temp["DEHST/NIR"]
df_facts["unit"] = ""
df_facts["rationale"] = ""
df_facts["reference"] = ""
df_facts["link"] = ""

In [ ]:
# df_facts.to_csv("facts_ratios_nir_to_dehst.csv", sep=",", index=False)

In [ ]:
# df_merged.to_csv("compare_excel_dehst.csv", index=False)

# Count

In [ ]:
df_anlagen[~pd.isna(df_anlagen["match_type"])].groupby("i_category").count()["Nummer"]

In [ ]:
df_anlagen[~pd.isna(df_anlagen["match_type"])][df_anlagen["i_category"] == "metal_steel_secondary"].sort_values("Ort")